In [1]:
from __future__ import print_function

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard,ReduceLROnPlateau

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
# from tensorflow.keras.datasets import mnist, cifar10
import tensorflow_addons as tfa

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scipy.io import loadmat 
import mat73
from datetime import datetime

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


print("Packages Loaded")

Packages Loaded


In [ ]:
# # Load data
# # raw_data1 = loadmat('new_echo_cnn_in_out_jstarrs2021_first_try/echo_cnn_in_out_jstars1.mat')
# # raw_data1 = loadmat('echo_cnn_in_out_GOOD_layers/new_echo_cnn_in_out_jstars1.mat') 'findpeaks_layers/new_echo_cnn_in_out_jstars1.mat'

# raw_data1 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars1.mat')
# raw_data2 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars2.mat')
# raw_data3 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars3.mat')
# raw_data4 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars4.mat')
# raw_data5 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars5.mat')
# raw_data6 = loadmat('findpeaks_layers_rowblock20/new_echo_cnn_in_out_jstars6.mat')

# d1 = raw_data1['echo_cnn1']
# t1 = raw_data1['echo_target1']
# i1 = raw_data1['echo_idx1']

# d2 = raw_data2['echo_cnn2']
# t2 = raw_data2['echo_target2']
# i2 = raw_data2['echo_idx2']

# d3 = raw_data3['echo_cnn3']
# t3 = raw_data3['echo_target3']
# i3 = raw_data3['echo_idx3']

# d4 = raw_data4['echo_cnn4']
# t4 = raw_data4['echo_target4']
# i4 = raw_data4['echo_idx4']

# d5 = raw_data5['echo_cnn5']
# t5 = raw_data5['echo_target5']
# i5 = raw_data5['echo_idx5']

# d6 = raw_data6['echo_cnn6']
# t6 = raw_data6['echo_target6']
# i6 = raw_data6['echo_idx6']

# all_data = np.concatenate( (d1,d2,d3,d4),axis = 0 )
# all_target = np.concatenate( (t1,t2,t3,t4),axis = 0 )
# all_idx = np.concatenate( (i1,i2,i3,i4),axis = 0 )



In [3]:
# base_path = os.path.join (os.getcwd() + '/all_block_data/Dec_Train_block_len_21_011121_2331/') # CHANGE HERE <==
base_path = '../all_block_data/Old_data'

# Confirm path is right...
print(f'{os.path.isdir(base_path)}')

True


In [5]:
# # Load data
# # raw_data1 = loadmat('Sept21/echo_cnn_in_out_jstars1.mat')

# raw_data1 = loadmat(base_path + '/new_echo_cnn_in_out_jstars1.mat')
# raw_data2 = loadmat(base_path + '/new_echo_cnn_in_out_jstars2.mat')

# d1 = raw_data1['echo_cnn1_reduced']
# t1 = raw_data1['echo_target1_reduced']
# d2 = raw_data2['echo_cnn2_reduced']
# t2 = raw_data2['echo_target2_reduced']
# all_data = np.concatenate( (d1,d2),axis = 0 )
# all_target = np.concatenate( (t1,t2),axis = 0 )




In [4]:
# raw_data1 = mat73.loadmat('all_block_data/Full_block_len_65_121021_0916/echo_cnn_in_out_jstars_65x15.mat')
raw_data1 = mat73.loadmat(base_path + '/Dec_Train_block_len_21_231121_1531/echo_cnn_in_out_jstars.mat')

In [5]:
all_data = raw_data1['echo_cnn_input']
all_target = raw_data1['echo_cnn_target']


In [6]:
row_length = 21 # CHANGE HERE <== 21,  16
col_length = 15

# Check that the dimension of data is correct
if all_data.shape[1] == row_length*col_length:
    print('Dimensions match')
else:
    print(f' Row block length:{row_length} and col length:{col_length} does not match Data dimension:{all_data.shape[1]}') 
        

Dimensions match


In [11]:
max_class = int(max(np.unique(all_target)))
all_target[all_target == max_class ] = 0


shuffle = 1
if shuffle:
    random.Random(13).shuffle(all_data)
    random.Random(13).shuffle(all_target)
    #random.Random(13).shuffle(all_idx)

## Prep data
train_size = int(np.floor(0.7*len(all_target)));
test_size = int(np.round( 0.2* all_data.shape[0] ))
val_size = all_data.shape[0] -train_size - test_size


x_train = all_data[0:train_size,:]
x_test = all_data[train_size:train_size+test_size,:]
x_val = all_data[-val_size:,:]

y_train = all_target[:train_size]
y_test  = all_target[train_size:train_size+test_size]
y_val = all_target[-val_size:]

var_input_shape = x_train.shape[1] # 240 columns
num_classes = max_class # layers
# Convert labels to categorical orthonormal vectors

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)


In [17]:
data_augmentation = Sequential(
    [
        layers.experimental.preprocessing.Rescaling(scale=1.0 / 255),
        layers.experimental.preprocessing.RandomCrop(row_length, col_length),
        layers.experimental.preprocessing.RandomFlip("vertical"),
        layers.experimental.preprocessing.RandomFlip("horizontal")
    ],
    name="data_augmentation",
)

In [12]:
# Create your own ResNet here.

shp = x_train.shape[1]

# Skip 3 connection version of base model
drop_out = 0.3
inputs = tf.keras.Input(shape=(shp,), name='img')
x = Dense(512, activation='relu')(inputs)
block_1_output = Dense(512, activation='relu')(x)

x = Dense(512)(block_1_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)

block_2_output = tf.keras.layers.add([x, block_1_output])

x = Dense(512)(block_2_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_3_output = tf.keras.layers.add([x, block_2_output])

x = Dense(512)(block_3_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_4_output = tf.keras.layers.add([x, block_3_output])

x = Dense(512)(block_4_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_5_output = tf.keras.layers.add([x, block_4_output])


x = Dense(512)(block_5_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_6_output = tf.keras.layers.add([x, block_5_output])

x = Dense(512)(block_6_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_7_output = tf.keras.layers.add([x, block_6_output])

x = Dense(512)(block_7_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_8_output = tf.keras.layers.add([x, block_7_output])

x = Dense(512)(block_8_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_9_output = tf.keras.layers.add([x, block_8_output])

x = Dense(512)(block_9_output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(drop_out)(x)
x = Dense(512)(x)
x = BatchNormalization()(x)

block_10_output = tf.keras.layers.add([x, block_9_output])


x = Dense(512, activation='relu')(block_10_output)
x = Dropout(drop_out)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name='resnet')

model.compile( Adam(amsgrad=True),loss= 'categorical_crossentropy', metrics=['accuracy'])  #tf.keras.losses.KLDivergence()

earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
# mcp_save = ModelCheckpoint('jstars_june_{epoch:02d}_{val_loss:.2f}.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=0, emin_delta =1e-4, mode='min')


model.fit(x_train, y_train,
          epochs= 100, 
          batch_size= 64, 
          validation_data=(x_test, y_test), callbacks=[ reduce_lr_loss]) #mcp_save,



Epoch 1/100
13705/13705 [==============================] - 5889s 428ms/step - loss: 2.5499 - accuracy: 0.1134 - val_loss: 2.5281 - val_accuracy: 0.1140
Epoch 2/100
13705/13705 [==============================] - 5897s 430ms/step - loss: 2.5282 - accuracy: 0.1146 - val_loss: 2.5277 - val_accuracy: 0.1138
Epoch 3/100
13705/13705 [==============================] - 5910s 431ms/step - loss: 2.5273 - accuracy: 0.1146 - val_loss: 2.5276 - val_accuracy: 0.1139
Epoch 4/100
13705/13705 [==============================] - 5887s 430ms/step - loss: 2.5269 - accuracy: 0.1155 - val_loss: 2.5276 - val_accuracy: 0.1164
Epoch 5/100
13705/13705 [==============================] - 5911s 431ms/step - loss: 2.5266 - accuracy: 0.1150 - val_loss: 2.5272 - val_accuracy: 0.1164
Epoch 6/100
13705/13705 [==============================] - 5901s 431ms/step - loss: 2.5265 - accuracy: 0.1153 - val_loss: 2.5272 - val_accuracy: 0.1139
Epoch 7/100
13705/13705 [==============================] - 5894s 430ms/step - loss: 2.52

KeyboardInterrupt: 

In [ ]:
# Train further

model.fit(x_train, y_train,
          epochs= 50, 
          batch_size= 64, 
          validation_data=(x_test, y_test), callbacks=[ reduce_lr_loss]) #mcp_save,

model.save('jstars_weight_21_15_filtered_image_july2021.h5')

In [ ]:

model.load_weights(mcp_save)
_, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")


model.save(f'{base_path}//ResNetCNN_weights//{time_stamp}_Acc_{accuracy}_Top5_{top_5_accuracy}_{row_length}x{col_length}.h5')

In [ ]:
# Test new model on val data

# y_pred = [np.argmax(model.predict(np.reshape(x_val[idx,:],(1,var_input_shape)))) for idx in range(len(x_val)) ]

y_pred = [np.argmax(model.predict(np.expand_dims(x_val[idx],axis=0))) for idx in range(len(x_val)) ]

combined_result = zip(y_val,y_pred)

exact_accuracy = 100 * sum([1 if y_pred[idx]==y_val[idx]   else 0 for idx in range(len(y_pred)) ]) / len(y_pred)
test_5_pixel_margin_accuracy = 100 * sum([1 if abs(y_pred[idx]-y_val[idx]) < 5 else 0 for idx in range(len(y_pred)) ]) / len(y_pred)

print(f' Exact accuracy is {exact_accuracy: .5f} %')
print(f' Top 5% accuracy is {test_5_pixel_margin_accuracy: .5f} %')



In [ ]:
# Test on training data

y_pred = [np.argmax(model.predict(np.expand_dims(x_train[idx],axis=0))) for idx in range(10000) ]

combined_result = zip(y_val,y_pred)

exact_accuracy = 100 * sum([1 if y_pred[idx]==y_train[idx]   else 0 for idx in range(len(y_pred)) ]) / len(y_pred)
test_5_pixel_margin_accuracy = 100 * sum([1 if abs(y_pred[idx]-y_val[idx]) < 5 else 0 for idx in range(len(y_pred)) ]) / len(y_pred)

print(f' Exact accuracy is {exact_accuracy: .5f} %')
print(f' Top 5% accuracy is {test_5_pixel_margin_accuracy: .5f} %')

In [ ]:
# Load saved model
# loaded_model = tf.keras.models.load_model('ResNet500_epoch200_batch64.h5')
loaded_model = tf.keras.models.load_model('all_block_data/Full_block_len_45_280921_1530/full45x21_echo_Sept2021.h5')


In [ ]:
# Predict using saved_model

y_pred = [np.argmax(loaded_model.predict(np.reshape(x_val[idx,:],(1,var_input_shape)))) for idx in range(len(x_val)) ]
combined_result = zip(y_val,y_pred)

exact_accuracy = 100 * sum([1 if y_pred[idx]==y_val[idx]   else 0 for idx in range(len(y_pred)) ]) / len(y_pred)
test_5_pixel_margin_accuracy = 100 * sum([1 if abs(y_pred[idx]-y_val[idx]) < 5 else 0 for idx in range(len(y_pred)) ]) / len(y_pred)

print(f'Loaded_model: Exact accuracy is {exact_accuracy: .5f} %')
print(f'Loaded_model: Top 5% accuracy is {test_5_pixel_margin_accuracy: .5f} %')

In [ ]:
model_predictions = np.asarray(y_pred)
ground_truth = y_val.squeeze()

In [ ]:
# Confusion matrix
confusion_mtx = np.zeros((num_classes,num_classes))

for pred,actual in zip(y_pred,y_val):
    confusion_mtx[int(pred),int(actual)] +=1

In [ ]:
# (A) Case where prediction was zero
zero_predictions_vs_gt = np.asarray( list(zip( model_predictions[model_predictions == 0], ground_truth[model_predictions == 0])) )
# (A) (ii)Case where prediction was zero but ground truth is different
zero_predictions_wrong = ( sum( [1 if iter[0] !=iter[1] else 0 for iter in zero_predictions_vs_gt ] ) ,len(zero_predictions_vs_gt) )
   
#(B) Case where ground truth was zero 
zero_gt_vs_predictions =  np.asarray( list(zip( ground_truth[ground_truth == 0], model_predictions[ground_truth == 0] )) ) 
#(B) (i) Case where ground truth was zero but prediction was different
zero_gt_right_pred= ( sum( [1 if iter[0] ==iter[1] else 0 for iter in zero_gt_vs_predictions ] ) ,len(zero_gt_vs_predictions) )



In [ ]:
# Prediction per Ground-truth class
class_predictions = [ model_predictions[ground_truth==idx]  for idx in range(num_classes) ]

# RMSE without the zero class for each class
named_class_rmse = [('Class-%s'%(each_class),np.sqrt (np.mean((each_class - class_predictions[each_class])**2)) ) for each_class in range(1,17) ]
per_class_rmse = np.asarray( [ np.sqrt (np.mean((each_class - class_predictions[each_class])**2)) for each_class in range(1,17)]  )  


In [ ]:
# Weight of each class
(_,cl_count) = np.unique(ground_truth,return_counts = True)
weight = cl_count/len(ground_truth)

total_rmse = np.mean( per_class_rmse* weight[1:] )



In [ ]:
# Histogram of each class prediction
for idx,each_class in enumerate(class_predictions):
    plt.figure()
    plt.hist(each_class, bins=np.arange(-0.5,17.5,1))
    plt.title('Class %s'%idx)

In [ ]:
def PlotCurves(model):
    
    plt.plot(model.history.history['accuracy'],label = 'train')
    plt.grid(color = 'r', linestyle = '--', linewidth = .1)
    plt.ylabel('Accuracy');
    plt.title('Training accuracy');
    plt.figure()
    plt.plot(model.history.history['val_accuracy'],label = 'validation')
    plt.grid(color = 'r', linestyle = '--', linewidth = .1)
    plt.ylabel('Val Accuracy');
    plt.title('Validation accuracy');

    return None

PlotCurves(model)

In [ ]:
# Evaluate on test set ( this should be evaluation set actually)
y_test1  = all_target[train_size:]
predictions = [(np.argmax(loaded_model.predict(np.reshape(x_test[idx,:],(1,240)))),int(y_test1[idx])) for idx in range(len(x_test)) ]



In [ ]:
# Plot to show some of the images of validation data

for iter_idx in range(150,160):
    plt.figure(figsize=(16,14));    
    plt.imshow( np.reshape(x_val[iter_idx],(45,-1),order='C'), cmap='gray' );
    plt.plot(y_val[iter_idx]*np.ones(15));
    plt.plot( np.ones(15) * np.argmax(loaded_model.predict(np.reshape(x_val[iter_idx,:],(1,var_input_shape)))),'ro' );
    plt.show()
    